# Working with Planet Data a.k.a Programmatic Adventures in Image Filtering

The aim of this Notebook is to help familiarize you with ways to start filtering Plant data using their API. This notebook is divided into two parts. In the first part, we will focus on the first phase of a larger research project: you have picked a few small study areas, and are likely wanting to build a time series of images for an area that you'll usef for classification, machine learning, deriving phenology, etc. Dealing with the challenges of identifying suitable images for your pilot study area is the focus of the first part of this notebook.

In the second part of this notebook, we will look at the second phase of a larger study. Having conducted a proof of concept study for several locations, you are now wanting to apply your methods to a larger study area. While the filtering techniques are similar, there are some different challenges you will face when working with larger spatial extents. Dealing with those challenges is the focus of the second half of the notebook.

Let's dive in!

This document is adapted from Planet's `search_and_download_quickstart.ipynb`

## Note - Why programmatic image filtering

There are many reasons for starting to deal with your data programmatically. One good reason is that it helps contributes to both open and repeatable science! That is certainly a worthy and worthwhile endevour. On the the practical side of research: when the volume of data is so large, working programmatically is the only way you will manage to be effective and efficient with your research. Why?

[Welcome to the world of constellations of satellite platforms and sensors](https://www.planet.com/our-constellations/).



## Part 1 - Image filtering for smaller areas

First step we are going to load the libraries we will be working with today. Hopefully that is straightforward. If not, ask

### import libraries

In [ ]:
import geojson
import json
import time

import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio

import requests
from requests.auth import HTTPBasicAuth
from shapely.geometry import Point, Polygon, MultiPolygon, shape

import matplotlib.pyplot as plt

import yaml

from pathlib import Path
import pathlib

### set working paths

In [ ]:
paths = {
    'iDir' : 'boundaries/',
    'oDir' : 'imagery/'
}

### define some helper functions

In [ ]:
def open_geojson(file_path):
    '''Function to open a GeoJSON file given a path.'''
    with open(file_path) as f:
#         gj = geojson.load(f)
        gj = json.load(f)
        
    return gj

def get_geojson_geometry(geojson):
    '''Function to extract the geometry attribute from a GeoJSON object '''
    geometry = [i['geometry'] for i in geojson['features']]
    
    return geometry

def plot_wrapper(gdf, ax, color, linewidth=1.5):
    '''Convenience function for overlaying spatial data on the same plot'''
    gdf.plot(
        facecolor="none",
        edgecolor=color,
        linewidth=linewidth,
        ax = ax   
    )

## Part 1 - Filtering for a pilot area of interest (AOI)

The steps we're going to follow here are:
   1. Load the data for our AOI
   1. Convert the data to a GeoDataFrame
   1. Examine the structure of a Planet API query
   1. Load results of a previously query 
   1. Convert results to a GeoDataFrame
   1. Filter! 

### Load the AOI data - GeoJSON

A GeoJSON is one of the many geospatial data formats. It is a more modern format, that many geospatial analysts are not familiar with. In some ways, it's broadly similar to an ESRI shapefile.

One noticable difference between a shapefile and a GeoJSON is the cooridnate reference system and projections. Whereas shapefiles support may different CRS and projections, GeoJSON only supports geographic coordinates. Yes, that is right. WGS84 Geographic is only CRS and coordinate system GeoJSON supports.
Western Minnesota study area

The area we will be using for this Notebook is focused on the western areas of Minnesota. This is farm country, and the GeoJSON contains two small farm fields that represnt the Area of Interest (AOI) for this study.

But first, here is a bit of an exploration of the GeoJSON format. Once imported, you'll note that its a MultiPolygon.

In [ ]:
west_mn = open_geojson('./' + paths['iDir'] + 'western_mn.geojson')
west_mn = get_geojson_geometry(west_mn)
west_mn

In [ ]:
geoms = [Polygon(west_mn[0]['coordinates'][i][0]) for i in range(len(west_mn[0]['coordinates']))]

aoi = gpd.GeoDataFrame(
    {'fields' : ['51', '46'], 'vals' : [10, 20] },
    geometry = geoms,
    crs = 4326
)
aoi

In [ ]:
aoi.explore()

### Structure of Planet query

Some key componets of the a query to the Planet API:
   1. API key. Basically this is your authorization credential for accessing and downloading data.
   1. Base URL. You interface with the API via the web. The base URL is the site you are interacting with
   1. Filter paramters. Several are required, and are likely familiar:
       1. Geometric filter - Spatially constrain the search
       1. Date & time filter - Temporal constraints on the search
       1. Cloud filter - Constrain results using cloud thresholds
       1. Combined filter - A filter that combines the above filters using boolean logic
       1. Item type filter - The Planet Image product you are after
       
       
Anatomy of a query:

#### API Key
Provided by Planet. Keys are usually a unique sting:
    e.g. key : `123veryweakkey456`

#### Geometric filter
```
geojson_geometry = west_mn[0]

# get images that overlap with our AOI 
geo_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}
```

#### Date & time filter
```
# get images acquired within a date range
date_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2021-05-25T00:00:00.000Z",
    "lte": "2021-05-25T23:59:59.000Z"
  }
}
```

#### Cloud filter
```
cc_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.0
  }
}
```

#### Combined Filter
```
combined_filter = {
  "type": "AndFilter",
  "config": [geo_filter, date_filter, cc_filter]
}
```

#### Item type
```
item_type = "PSScene"
# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}
```


### Putting the query together:
```
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)
```

## Processing seach return

Planet will return a JSON object. Let's load the results of the query and have a look at it.

In [ ]:
with open('./data/pilot_query.json', 'r', encoding='utf-8') as f:
    results_json = json.load(f)

results_json

Convert this to a DataFrame. JSON is good for transit across the web, but not for analysis.

In [ ]:
tiles = pd.json_normalize(results_json['features'], max_level=1)

# rename properties column, dropping the properties label
prop_cols = {col: col.split('.')[1] for col in tiles.columns if 'properties' in col}
tiles.rename(columns=prop_cols, inplace=True)
tiles.info()

### Convert the results to a GeoDataFrame
Getting to a DataFrame is useful. Some of the filtering we are going to do now can be accomplished using the metadata alone. More useful however, will be to make this a GeoDataFrame. Once we have a GeoDataFrame, we will then be able to make use of spatial operations as well as traditional filtering.

In order to use some additional filtering logic, we're going to convert the information about the returned tiles to a geopandas GeoDataFrame. To do that, we need to add information about the geometry. That was returned to us in the JSON object (geometry.coordinates) and all we need to do is tell geopandas about the geometry. In this case it's a Polygon, so we'll use the Polygon object from shapely to define it.

In [ ]:
geoms = [Polygon(geo[0]) for geo in tiles['geometry.coordinates']]
tiles = gpd.GeoDataFrame(tiles, geometry = geoms, crs=4326)
tiles['centroids'] = tiles['geometry'].centroid
tiles.head()

In [ ]:
ax = tiles.explore( color ='orange', style_kwds={'stroke': True, 'fill': False})
aoi.explore( color= 'blue', m = ax)

Let's have a look at what sensor types were returned.

In [ ]:
instruments = set(tiles['instrument'])
instruments

### Sensor type

Over time, Planet has improved and evolved the sensors comprising their satelite constilations. Depending on what sensors were passing over, you may find that sensor type is able to reduce the number of tiles. As can be seen, there are two sensor types returned in the above: 'PS2.SD', 'PSB.SD'. These types are described in detail by Planet.

Of revlevence the PS2.SD is the second generation of the Planet Scope line, and PSB.SD is the third generation of their Planet Scope Sensor. Depending on the time frame for your study, not all of the sensor types will be available, and may not provide a useful filtering option.

You'll see in the above, there is only one PS2.SD tile, but since that is an older generation sensor, it would be a decent option if it were the ony tile availabe. But seeing as there are 18 tiles from the newer generation, we filter on PSB.SD and then look at other options for filtering down.


In [ ]:
tiles = tiles[tiles.instrument=='PSB.SD']
tiles.info()

###  Find tiles that contain the AOI

Alhough there are a lot of footprints in the figure above, particularly for the PS2.SD insturmnets, if you look at the left most of the blue AOI polygons, you will notice that it is bisected by several of the footprints. One way to filter tiles down is to identfy only those tiles that contain the AOI.

Looking for tiles that contain an AOI will not always prove useful. In the next section the focuses will be on larger study areas, the AOI is large, spaning several different satellite overpass tracks. Containment by tile footprints will not be useful in that instance. If you have several locations that are close together (as in this instance), it can be useful. Having all of your close together AOIs observed by a single sensor means can help ensure variablity in the AOIs is a function of actual variablity. In the older generations of sensors, there was much more radiometric variability in the insturments, and having all of your AOIs in one tile helped to ensure that variability in the AOI was a function of what was happening on the ground, and not an artifact of being observed by different platforms.


In [ ]:
tiles_contain_aoi = gpd.sjoin(tiles, aoi,  how='inner', predicate='contains' )
# tiles_contain_aoi = gpd.sjoin(aoi, tiles, how='inner', predicate='within')

# tiles_contain_aoi = tiles.sjoin(aoi, how='inner', predicate='contains')
tiles_contain_aoi.info()

In [ ]:
tiles_contain_aoi.shape

Now, instead of a data frame with 18 tiles, we have one with 27! What is going on?

Basically, since the AOI is composed of multiple polygonds, each tile has to determine whether or not it containes each of the AOI shapes by itself. a check for whether or not it contains the shape independently. What we end up with is a dataframe that has repeated rows. That is, the result after sjoin is we have all of the rows that either contained either the left field, or the right field.

What we want to do is identify the fields where both left and right were contained. This becomes a bit clearer, when we look at the indexex of the resulting dataframe. Notice that their are repeated index values?


In [ ]:
tiles_contain_aoi.index

We can use the `duplicated()` method to identy which ones are repeated. By using the `keep=False` we can see which of the values is not repeated. So looks like there is only one value that is not repeated. This indicates that of all the Planet tiles that contain the left field, only one does not also contain the right field.

We can use this fact to generate the unique set of fields that contain both fields


In [ ]:
tiles_contain_aoi.index.duplicated(keep=False)

In [ ]:
idx = tiles_contain_aoi.index.duplicated(keep=False)
idx = list(set(tiles_contain_aoi.loc[idx].index))
idx, len(idx)

We can now see, that of our 18 tiles captured by our preferred Planet constillation, there are 13 tiles that contain both fields. To confirm this we'll use the index values to regenerate the `tiles_contain_aoi` to represent only the unique tiles that actually contain both fields.

Before we do that, we will use the indexes of the contained tiles to show that this list is only tiles that contain both fields

In [ ]:
for i in idx:
    ax = tiles[tiles.index==i].plot(edgecolor='orange', facecolor='none')
    aoi.plot(edgecolor='blue', facecolor='blue', ax=ax)

And we can also use the indexes to plot the tiles that didn't contain both, just as a double check

In [ ]:
for i in tiles[~tiles.index.isin(idx)].index:
    ax = tiles[tiles.index==i].plot(edgecolor='red', facecolor='none')
    aoi.plot(edgecolor='blue', facecolor='blue', ax=ax)

Now, we'll use those indexes to subset the tiles dataframe so that it only contains those tiles that contained both fields. This is preferable to working with the original tiles_contain_aoi generated with the `.sjoin()` method, as it will not contain duplicated tiles. When it comes to actually ordering and downloading tiles from Planet, the aim is to avoid duplication.

In [ ]:
tiles_contain_aoi = tiles[tiles.index.isin(idx)]
tiles_contain_aoi.info()

Thirteen images is still a lot of duplicated data. While we might want a such a dense time series if we were wanting to examine the ways in which satellite overpass, view angle angle, and geometry of incident solar radiation impacted the observation values, this might be ok.

### Acquistion time

But in this case, we'll use the aquisition time to filter out additional files. To do so, we will have a closer look at the acquired field.

Ideally, what we would like are images that were aquired close to solar noon. But in looking at those dates, it seems we have a problem in that the hour are 16, 17 which look to be potentially ~4 - 5 hours after noon. If that were true, it would not be very convenient.

Note that the `dtype` of the column is `object`. What we really want is a `datetime` object. Also note that the values all end in a `Z`. This is in part because the Planet data are collected in UTC time, which means that the apparently 4-5 hour difference between solar noon is wrong. So, let's set change this to a `datetime` object, and then change the timezone


In [ ]:
tiles_contain_aoi['acquired']

In [ ]:
tiles_contain_aoi['acquired'] = pd.to_datetime(tiles_contain_aoi['acquired'], utc=True)
tiles_contain_aoi['acquired'] = tiles_contain_aoi['acquired'].dt.tz_convert('US/Central')
tiles_contain_aoi['acquired']

During the time of year that we're exploring, UTC is 5 hours ahead of US/Central time. To idenfity the image closest to solar noon, we'll need to create a time variable for comparison.

To do that, we will use the date portion of the Datetime object from the first row in our dataframe. Since this dataframe only has images from a single date, we could have manually set the `solar noon` variable in this instance. However, this code block below shows you how to set it semi-programatically. That is, the date portion of the object is extracted from an entry in the table, and the time and time zone are then hardcoded/

There are likely ways to set the time programatically, depending on your locatation. Hopefully, you are not trying to do this for AOIs that span multiple time-zones. If you are, you probably want to rethink your filtering approach. Even if you set both the date and time programmatically, it's likely to become messy trying time to filter your tile list down when dealing with areas that span multiple time zones.


In [ ]:
solar_noon = tiles_contain_aoi.loc[0]['acquired'].strftime("%Y-%m-%d")
solar_noon = pd.to_datetime(f'{solar_noon}T17:00:00', utc=True).tz_convert('US/Central')
# solar_noon = pd.to_datetime(np.datetime64(f'{solar_noon}T15:00:00'))#.dt.tz_conver('US/Central')
solar_noon#.dt.tz_localize('UTC')

In [ ]:
time_diff = tiles_contain_aoi['acquired'] - solar_noon
np.abs(time_diff)

In [ ]:
min_idx = time_diff[np.abs(time_diff) == np.abs(time_diff).min()].index
min_idx

In [ ]:
tiles_contain_aoi.loc[min_idx]['id'].to_list()

In [ ]:
tiles_contain_aoi.loc[min_idx]['assets'].to_list()

## Part 2 - Filtering for large spatial exents

Some of the steps in the next section of the notebook will look familiar. We just practiced them above. However, working with larger spatial extents has some specific challenges. We will explore those in this section.

#### Twin Cities Metro Area

The area we will be using for this section is focused on the Twin Cities Metro area. We'll start by loading the file. Once loaded, we'll do a simple spatial transformation in order to give you a sense of how much area it covers.

But first, here is a bit of an exploration of the GeoJSON format. Once imported, you'll note that its a Polygon, and it's basically a rectanglar area in geographic coordinates

In [ ]:
tc_metro = open_geojson('./' + paths['iDir'] + 'twin_cities_metro.geojson')
tc_metro['features']


In [ ]:
tc_bounds = get_geojson_geometry(tc_metro)
tc_bounds

#### Convert the GeoJSON to GeoDataFrame


In [ ]:
aoi = gpd.GeoDataFrame({'name' : ['Twin Cities Metro']}, geometry = [Polygon(tc_bounds[0]['coordinates'][0])], crs = 4326)

In [ ]:
aoi.explore(color='blue', style_kwds={'stroke': True, 'fill': False})

Let's be a bit quantitative and get an idea of the actual dimensions we are looking at

In [ ]:
aoi_utm = aoi.to_crs(32615)
aoi_utm.area[0] /1000**2

In [ ]:
length = (aoi_utm.bounds['maxx'][0] - aoi_utm.bounds['minx'][0]) / 1000
width = (aoi_utm.bounds['maxy'][0] - aoi_utm.bounds['miny'][0]) / 1000
print(f'length: {length}')
print(f'width: {width}')
print(f'area: {length * width}')

### Pause to consider: Magnitude of the area vs amount of available data
efore we get into the actual querrying and filtering aspect of working with the AOI, we really do want you to pause and consider that area above. The area for our study is ~$9,480 km^{2}$. Let's just round up and call it $10,000 km^{2}$.

The reason this notebook exits, is to help you understand the importance of filtering data down to managable sizes. Using the query and the data we're going to work with in a few minutes, is entirely possibly to download $10x$ the required area using a single day's worth of imagery.

Yes, that is right. Without filtering down the data, you can without much trouble, end up with $~ 100,000 km^{2}$ of data. of which you didn't really need. Not only do you not need it, queries like this can consume significant amounts of UMN's quota. We ask that you be conscious of that when ordering data.

### Query details

#### Geometry
```
geojson_geometry = tc_bounds[0]
# get images that overlap with our AOI 
geo_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}
```

#### Date & time
```
# get images acquired within a date range
date_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2021-06-12T00:00:00.000Z",
    "lte": "2021-06-13T00:00:00.000Z"
  }
}
```

#### Cloud filter
```
cc_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.0
  }
}
```

#### Combined filter
```
# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geo_filter, date_filter, cc_filter]
}
```

#### Item/ Asset Filters
```
item_type = "PSScene"
# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}
```


### Search query
```
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)
```

### Load the search results

The query above was run in advance for you. Now, let's have a look at it. As before, it won't be super useful in this format. So we'll go through the process of transforming it as before.

In [ ]:
with open('./data/large_area_query.json', 'r', encoding='utf-8') as f:
    results_json = json.load(f)

results_json

In [ ]:
tiles = pd.json_normalize(results_json['features'], max_level=1)

# rename properties column, dropping the properties label
prop_cols = {col: col.split('.')[1] for col in tiles.columns if 'properties' in col}
tiles.rename(columns=prop_cols, inplace=True)
tiles.info()

### Note
This notebook was originally developed, in part, because cloud filtering turned out not to be useful on this partiulcar day. Note above the cloud filter is quite agressisve less than or equal to .0. Relaxing this constraint to allow for 50% cloud cover will not result in fewer tiles being identified. 

This is an important difference between _constillations sensor_ of satellites and single platform sensors. Between days, cloud cover varies significantly. Within a given day, cloud cover can vary, but sometime it doesn't vary much at all. That is what you are seeing here - this was a very clear day across the Cities!

### Geodataframe
The GeoDataFrame will look similar to the DataFrame, but in this case a geometry column and a `centroids` column have been added


In [ ]:
geoms = [Polygon(geo[0]) for geo in tiles['geometry.coordinates']]
tiles = gpd.GeoDataFrame(tiles, geometry = geoms, crs=4326)
tiles['centroids'] = tiles['geometry'].centroid
tiles.head()

In [ ]:
tiles.set_geometry('geometry', inplace=True)

In [ ]:
ax = tiles.explore( color ='orange', style_kwds={'stroke': True, 'fill': False})
aoi.explore( color= 'blue', m = ax, style_kwds={'stroke': True, 'fill': False})
tiles['geometry'].centroid.explore(color='k', m =ax)

That's a very busy, image. So let's take a bit of time to examine it, and look at some details more closely and discuss them.

First, the tile foot prints are larger than the AOI. The Planet API geometry filter uses an `intersection` for the geometry, meaning it will return any tile that touches the AOI. It would be nice if they supported other spatial operations, but they don't. Filtering using other spatial operations has to be done on the user side.

You may note that the figure is really busy. If you realized that there are a number of different sized tiles in that figure you are right. Though the data are in geographic cooridnate (and hence the `.area` method will spit out a warning, we can get a sense of the relative size differences in the footprints.


In [ ]:
tile_area = tiles['geometry'].area
tile_area.values

Though the units are not helpful, we can see that tiles vary by factors of
$2x$ and $4x$ . We can use that basic information to look at those differences

In [ ]:
ax = aoi.explore( color= 'blue', m = ax, style_kwds={'stroke': True, 'fill': False})
# biggest tiles first
idx = tiles['geometry'].area >= 0.079
tiles.loc[idx].explore( color='orange', m = ax, style_kwds={'stroke': True, 'fill': False})

# medium tiles next
idx = ((tiles['geometry'].area >= 0.04) & (tiles['geometry'].area < 0.079) )
tiles.loc[idx].explore(color='green', m = ax, style_kwds={'stroke': True, 'fill': False})

# small tiles next
idx = (tiles['geometry'].area < 0.04) 
tiles.loc[idx].explore( color='black', m = ax, style_kwds={'stroke': True, 'fill': False})

AOI is in blue still, the larger tiles are in orange, the smallest in grey, and the medim tiles are in green. These differences in footprint likely correspond with different generations of Planet sensor constillations. This figure makes it clear, there are not as many of the medium resoltuion foot prints, and the sensor with the smaller foot prints. Neither the small nor medium has a good spatial coverage of the AOI.

This suggests that the area of the footprint can be used to filter this list down! Even though the units are not meaningful ($decimal degrees ^{2}) they still provide a relative sense of area that we can use to filter the list down.

Let's do that now.


### Constillation sensor type
Before filtering based on tile footprint, it's worth discussing the reason for these footprint differences. As noted above, Planet operates constillations of satellites. Over time, they have improved their sensor platforms. Some of these changes include increased radiometric performance (allowing sensor values to be comperable between individual sensors within the constillation family), some have increased the number of spectral bands (allowing for increased discrimination of land cover types), and other changes have changed the band-pass characteristics (allowing for better comparison with other satellite platforms, such as Landsat and Sentinel).

Improvements in some sensor characteristics often invovles a trade off with other characteristics, and trading off tile footprint. Footprint is a function of the instanteous field of view IFOVof the sensor. So, before moving on, let's have a look at the different sensor types with the images returned in this query.

In [ ]:
instruments = set(tiles['instrument'])
instruments

In [ ]:
colors = ['orange', 'green', [.4,.4,.4] ]

for instrument, color in zip(instruments,colors):
    fig, ax = plt.subplots(1, 1, figsize=(15,15))

    aoi.plot(facecolor='none', edgecolor='blue', linewidth = 2, ax = ax)
    plot_wrapper(tiles[tiles.instrument == instrument], ax = ax, color=color)
    fig.suptitle(f'Tile footprints for sensor type: {instrument}', fontsize=20)
    plt.tight_layout()

We can see there is a relationship between tile footprint and sensor generation, so let's have a quick look at what the 'area' of each tile footprint is for each sensor type.


In [ ]:
for instrument in instruments:
    print(f'instrument type: {instrument}')
    print(tiles[tiles.instrument==instrument].area)
    print(f'\n')

### Examine the `PSB.SD` sensors, AKA 'largerest' tiles to start

We could use tile size directly, and an early version of this Notebook did just that. The relationship between footprint and instrument type are pretty clear. If you understand that relationship, you can use it to your advantage. Maybe you want only the largest footprints for a given sensor type. Understanding your study needs may point you in a particular direction.

Here, we're going to use the relationship between sensor type and footprint to select the largest tiles. In addition to having a relationship with footprint, there are good reasons filter on instument type. As noted above, [sensors have improved over time](https://earth.esa.int/eogateway/documents/20142/37627/Planet-combined-imagery-product-specs-2020.pdf), and those improvements have led to better radiomentric image quality in particular. It is worth making an [effort to understand sensor characteristcs](https://www.mdpi.com/2072-4292/13/19/3930), as this can have a large impact on the quality and veracity of your study results.

In [ ]:
tiles.set_geometry('geometry', inplace=True)
# idx = tiles['geometry'].area >= 0.079

larger_tiles = tiles[tiles['instrument']=='PSB.SD']
larger_tiles.info()

You'll note that we've now cut the list of tiles down by more than half! We're down to $41$ tiles from $93$. It's a good start. We can also visualize the new set. In doing so, we'll see we've removed much of the spatial duplication that was there.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,15))

aoi.plot(facecolor='none', edgecolor='blue', linewidth = 2, ax = ax)
larger_tiles.plot(facecolor='none', edgecolor='orange', ax = ax)
larger_tiles['geometry'].centroid.plot(color='k', ax =ax)

You can see that the overlap between tiles is not uniform across the AOI, but it's better than it was. There is considerable overlap in tiles along the left hand edge of the AOI. Before we move on to ways to refine the filtering futher, I want to focus briefly on what Planet calls `Products` and what they call `Assets`.

Above, when we querried the API, we specifically asked for Planet's `PSScene` product. The section of code was:

```
item_type = "PSScene"
# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}
```
Most University license agreement with Planet gives us access to to Planet products `PSScence` and `Basemap`. `PSScence` represent their imagery products. The `Basemap` product is built from the `PSScence` imagery, and is basically a composited product. That is, the `Basemap` is basically a composit of all the best images captured during the time period in question. Our license agreement gives us access to both their monthly and quartery `Basemap` composites. In the terminology of Planet's API, those quarterly and monthly `Basemaps` are referred to as `Assets`.

Planet `Assets` are basically a sub-type of the `Product` class, that have particular characteristics. For the `Basemap` product, charcteristics of the `Assets` is reasonably self evident: the monthly `Basemap Asset` is a composite product that represents the best imagery from the month in question (e.g. June 2021) and the quarterly `Basemap Asset` that covers the June 2021 period would be composed of images captured between April - June 2021.

So, with that in mind, let's look at how many Planet Assets there are for some of the PSScene tiles we asked the API to retrieve.

In [ ]:
tiles.loc[0]['assets'], tiles.loc[34]['assets']

The two lists are not the same, but that is ok. The images come from two different generations of constillations. The first one is from their SuperDove constilation, which has 8 spectral bands (e.g. `ortho_analytic_8b_sr`, and the second is from an older generation, as it only has data from up to 4 bands `ortho_analytic_4b_sr`).

To help manage the fact that there are sensors with different numbers of bands, backwards compatability is maintened by the fact that you can basically get the 4 band subset (red, green, blue, NIR) from the newer 8 band constillation.

### Filtering take away :

The focus of this part is on filtering the number of tiles down. As part of this discussion, it's important to recognize that within the single `PSScence` product, there are a number different `Asset` types. Knowing which specific `Asset` type you are after is important, as if you download multiple Assets each individual Asset counts against the U of M Tile Download Quota.

So you can see, it not hard to end up with significantly more data than you need, and much of it will be redundant! For example, using the 8-band constilation, if you downloaded both the `ortho_analytic_4b_sr` and `ortho_analytic_8b_sr` `Assets` half of your data will be duplicates!

For most quantitative analyses, the suggestion would be to work with either the `PSScence` `ortho_analytic_4b_sr` or `ortho_analytic_8b_sr` `Asset`, denending on your needs. If you are looking back in time, understand that the 8-band sensors have only come online in recent years, and that the further back in time you search, you're most likely to find older generations of the 4-band sensors.

### Back to filtering
If you look back to the last image, you'll note that a number of the image tiles don't have centroids that fall within the AOI. You'll also note that those with centroids within the AOI tend to make up more of the area within the AOI.

This suggests we can use the spatial operation `within` as a way to filterdown the tile list further. So let's do that.

###  start with tiles with centroids contained by AOI

These will be tiles with more useable foot print areas within the AOI.

In [ ]:
larger_tiles.set_geometry('centroids', inplace=True)
# tiles_centroids_contained = gpd.sjoin(tiles, aoi , how ='inner', op='within')
# tiles_centroids_contained = aoi.sjoin(tiles, how ='right', predicate='contains')
lt_centroids_contained = gpd.sjoin(larger_tiles, aoi, how='inner', predicate='within' )
lt_centroids_contained.set_geometry('geometry', inplace=True)
lt_centroids_contained.info()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,15))

aoi.plot(facecolor='none', edgecolor='blue', linewidth = 2, ax = ax)
lt_centroids_contained.plot(facecolor='none', edgecolor='orange', ax = ax)
lt_centroids_contained['geometry'].centroid.plot(color='k', ax =ax)

So, you can see, we have a smaller set that comes close to being what we need. We see that there is still more area outside the AOI then we really want, but that is ok! Part of the Planet API functionality does include the ability to clip tiles using the AOI before are downloaded. We'll focus on that later in the Notebook.


### the contained centroids form basis of our 'to_download_set'

we'll make a copy to be on the safe side. we'll then append to this set using the areas not covered by these tiles shortly


In [ ]:
to_download = larger_tiles.copy()

### get larger tiles with centroid that are not contained

For now, let's look at the tiles with centroids not contained by the AOI. Clearly, if we want a complete AOI, we'll need to add some of them to our tile list. They will be needed to fill out the gaps around the edges of the AOI.


In [ ]:
lt_centroid_not_contained = larger_tiles[~larger_tiles.index.isin(lt_centroids_contained.index)]
lt_centroid_not_contained.set_geometry('geometry', inplace=True)
lt_centroid_not_contained.info()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,15))

aoi.plot(facecolor='none', edgecolor='blue', linewidth = 2, ax = ax)
lt_centroid_not_contained.plot(facecolor='none', edgecolor='green', ax = ax)
lt_centroid_not_contained['geometry'].centroid.plot(color='k', ax =ax)



So, we have a list of tiles we know we are going to use (those with centroids within the AOI) and a list of candidates to choose to include to fill out the edges. How to proceed next? Basically, we're going to leverage the geometry of the tiles with contained centroids. We will use those tiles to figure out which areas we need to fill.

We'll then use the 'blank areas' that we need to fill to find the tiles whose centroids were not contained by the AOI to identify the candiates for filling. We'll then iterate through the tiles that touch each 'blank area' to see how much of their gemetry fills the blank. Once part of the blank is filled, we'll continue until there are no tiles, or no blank left to fill!


###  use the geometry of contained centroids

now we'll use the geometry of the larger tiles to identify areas with no coverage in the AOI we'll start by creating a copy the larger tiles geodataframe, then we'll disolve the geomety into a single shape. we'll then clip the disolved shape using the AOI, and then use the geopandas difference to find areas not covered by the contained centroid tiles

In [ ]:
lt_disolved = lt_centroids_contained.copy()
lt_disolved = lt_disolved.dissolve()
lt_disolved = lt_disolved.clip(aoi)
lt_disolved.plot()

In [ ]:
aoi_missing = aoi.difference(lt_disolved)
aoi_missing.plot()

In [ ]:
aoi_missing

What we end up with is a MultiPolygon that has all of the areas that we need to fill. There are several parts, all of differing sizes.

If you're not familiar with a MutliPolygon, it is basically a Polygon that is made up of different Polygons. They can be complciated (think of a doughnut) but in this case, thankfully it's not complicated. We can use the `.explode` method to break that MultiPolygon into it's parts, then iterate though them piece by piece.

### use the parts of the resulting missing multi-polygon to identify intersecting tiles

In [ ]:
aoi_miss_parts = aoi_missing.explode()
aoi_miss_parts.info()

###  loop through the missing part areas to find larger tiles with centroids that were not contained that intersect the missing part

In [ ]:
additional_tile_ids =[]
for i in range(aoi_miss_parts.shape[0]):
    poly = aoi_miss_parts[0,i]
#     t = gpd.overlay(lt_centroid_not_contained, mp, how = 'intersection')
    ix = lt_centroid_not_contained.intersects(poly, align=False)
    
    t = lt_centroid_not_contained.loc[ix].copy()
    t = t.clip(poly)
    t['area'] = t['geometry'].area
    t.sort_values('area', ascending=False, inplace=True)
    
    for i,r in t.iterrows():
        
        if poly.is_empty:
            break 
            
        additional_tile_ids.append(r['id'])
        poly = poly.difference(r['geometry'])

Now that we've done the processing, let's see how many additional tiles it will take to fill in the blanks. For comparison, we'll also look at the original number of tiles with centroids that we not contained by the AOI.

In [ ]:
additional_tile_ids

In [ ]:
len(additional_tile_ids), lt_centroid_not_contained.shape[0]

So, we've saved ourself 9 tiles that we didn't need. Though some of the area taken up by those tiles is not large, we will also save ourselves the hassel of downloading duplicated data too, which is a win.

###  combine and visualize

As a last step, we'll use the additional_tile_ids list to add the tile to our


In [ ]:
to_download = larger_tiles[larger_tiles.index.isin(lt_centroids_contained.index)].copy()
to_download = pd.concat([to_download,larger_tiles[larger_tiles.id.isin(additional_tile_ids)]])
to_download.set_geometry('geometry', inplace=True)
to_download.info()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,15))

aoi.plot(facecolor='none', edgecolor='blue', linewidth = 2, ax = ax)
to_download.plot(facecolor='none', edgecolor='orange', ax = ax)
to_download['geometry'].centroid.plot(color='k', ax =ax)

We now have a list of 31 one tiles that can be used to represent the AOI. Although there isn't time today, ordering tiles is straightforward, and there are good resouces on how to do that. It's also worth noting that we could use Planet's `tool chain` to `clip` those  images down, so that all we get is just the area within the TC AOI. Their `tool chain` contains other useful image processing tools that might also be of interest.